In [5]:
import psycopg2
import pandas as pd
from psycopg2.extras import RealDictCursor
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
es = Elasticsearch()

In [2]:
def create_connection():
    return psycopg2.connect("dbname=twitter user=postgres password=postgres host=localhost port=5433")

Select z databazy. Podminka t.location IS NULL je z dovodu, ze vacsina tweetov nema location (+- 99%) a pri mapovani sposobovali tweety s hodnotou v location problem. Tiez som vybral iba extremne tweety, z dovodu setrenia miesta na disku.

In [3]:
select = """
SELECT t.*,
       a.name,
       a.screen_name,
       a.description,
       ARRAY_AGG (h.value) used_hashtags
FROM tweets AS t
JOIN tweet_hashtags AS th ON th.tweet_id = t.id
JOIN hashtags AS h ON th.hashtag_id = h.id
JOIN accounts AS a ON t.author_id = a.id
WHERE (t.compound > 0.5 OR t.compound < -0.5) AND t.location IS NULL
GROUP BY t.id, a.id
"""

conn = create_connection()
df = pd.read_sql_query(select, con=conn)
conn.close()

## Uloha 2, 3, 4 - vytvorenie mapovania a analyzerov

Pocet shardov je 3, pretoze mame 3 nody, tak aby na kazdom node bezal aspon 1 shard. Kazdy shard ma 2 repliky pre pripad, ze by node na ktorom bezi dany shard odisiel.

In [33]:
mapping = {
   "settings":{
      "index":{
         "number_of_shards":3,
         "number_of_replicas":2,
         "max_ngram_diff":9,
         "analysis":{
            "analyzer":{
               "englando":{
                  "filter":[
                     "english_possessive_stemmer",
                     "lowercase",
                     "english_stop",
                     "english_stemmer"
                  ],
                  "char_filter":[
                     "html_strip"
                  ],
                  "tokenizer":"standard"
               },
               "custom_ngram":{
                  "filter":[
                     "lowercase",
                     "asciifolding",
                     "filter_ngrams"
                  ],
                  "char_filter":[
                     "html_strip"
                  ],
                  "tokenizer":"standard"
               },
               "custom_shingles":{
                  "filter":[
                     "lowercase",
                     "asciifolding",
                     "filter_shingles"
                  ],
                  "char_filter":[
                     "html_strip"
                  ],
                  "tokenizer":"standard"
               }
            },
            "filter":{
               "english_possessive_stemmer":{
                  "type":"stemmer",
                  "language":"possessive_english"
               },
               "english_stop":{
                  "type":"stop",
                  "stopwords":"_english_"
               },
               "english_stemmer":{
                  "type":"stemmer",
                  "language":"english"
               },
               "filter_ngrams":{
                  "type":"ngram",
                  "min_gram":1,
                  "max_gram":10
               },
               "filter_shingles":{
                  "type":"shingle",
                  "token_separator":""
               }
            }
         }
      }
   },
   "mappings":{
      "properties":{
         "id":{
            "type":"text"
         },
         "content":{
            "type":"text",
            "analyzer":"englando"
         },
         "location":{
            "type":"point",
            "null_value": "0,0"
         },
         "retweet_count":{
            "type":"integer"
         },
         "favorite_count":{
            "type":"integer"
         },
         "happened_at":{
            "type":"date",
            "format":"yyyy-MM-dd HH:mm:ss"
         },
         "author_id":{
              "type":"long"
         },
         "country_id":{
            "type":"integer",
            "null_value": -1
         },
         "parent_id":{
            "type":"text"
         },
         "neg":{
            "type":"double"
         },
         "neu":{
            "type":"double"
         },
         "pos":{
            "type":"double"
         },
         "compound":{
            "type":"double"
         },
         "author":{
            "type":"nested",
            "properties":{
               "name":{
                  "type":"text",
                  "fields":{
                     "ngram":{
                        "type":"text",
                        "analyzer":"custom_ngram"
                     },
                     "shingles":{
                        "type":"text",
                        "analyzer":"custom_shingles"
                     }
                  }
               },
               "screen_name":{
                  "type":"text",
                  "fields":{
                     "ngram":{
                        "type":"text",
                        "analyzer":"custom_ngram"
                     }
                  }
               },
               "description":{
                  "type":"text",
                  "analyzer":"englando",
                  "fields":{
                     "shingles":{
                        "type":"text",
                        "analyzer":"custom_shingles"
                     }
                  }
               }
            }
         },
         "hashtags":{
            "type":"keyword",
            "normalizer":"lowercase"
         }
      }
   }
}

Vytvorenie indexu podla nastaveni a mapovania:

In [34]:
es.indices.delete(index='my_index', ignore=[400, 404])
es.indices.create(index='my_index', settings=mapping["settings"], mappings=mapping["mappings"])

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'}

## Uloha 5, 6 - vytvorenie bulk importu a naimportovanie prvych 5000 tweetov

In [43]:
bulk_ = []

for index, row in df.iterrows():
    if index == 5000:
        break
    c_id = -1    
    if not str(row['country_id'] == "nan"):
        c_id = int(row['country_id'])
    doc = {
    "_index": "my_index",
    'id': row['id'],
    'content': row['content'],
    'location': row['location'],
    'retweet_count': row['retweet_count'],
    'favorite_count': row['favorite_count'],
    'happened_at': str(row['happened_at']).split("+")[0],
    'author_id' : row['author_id'],
    'country_id':c_id,
    'parent_id': row['parent_id'],
    'neg': row['neg'],
    'neu': row['neu'],
    'pos': row['pos'],
    'compound': row['compound'],
    'author': {
        "name": row['name'],
        "screen_name": row['screen_name'],
        "description": row['neg'],
    },
    "hashtags" : row['used_hashtags']
    }
    bulk_.append(doc)

Kniznicna funkcia pre bulk:

In [39]:
bulk(es, bulk_)

(5000, [])

## Uloha 8 - upravenie poctu retweetov

Pomocou localhostu som si vyhladal prvy tweet a podla jeho _id som upravil pocet retweetov

In [42]:
es.update(index='my_index',doc_type='_doc',id="_I9Fr30B6vb7fMH_fvO4",body={"doc": {"retweet_count": 100 }})

C:\Users\Marek\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'update' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  """Entry point for launching an IPython kernel.


{'_index': 'my_index',
 '_type': '_doc',
 '_id': '_I9Fr30B6vb7fMH_fvO4',
 '_version': 2,
 'result': 'updated',
 '_shards': {'total': 3, 'successful': 3, 'failed': 0},
 '_seq_no': 4989,
 '_primary_term': 1}

## Uloha 9

Zrusenie replik chapem ako zmazanie prvych 5000 zaznamov importovanych v ulohe c. 6.

In [44]:
es.indices.delete(index='my_index', ignore=[400, 404])
es.indices.create(index='my_index', settings=mapping["settings"], mappings=mapping["mappings"])

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'}

In [48]:
bulk_ = []

for index, row in df.iterrows():
    if index % 5000 == 0:
        bulk(es, bulk_)
        bulk_ = []
    c_id = -1
    if not str(row['country_id'] == "nan"):
        c_id = int(row['country_id'])
    doc = {
    "_index": "my_index",
    'id': row['id'],
    'content': row['content'],
    'location': row['location'],
    'retweet_count': row['retweet_count'],
    'favorite_count': row['favorite_count'],
    'happened_at': str(row['happened_at']).split("+")[0],
    'author_id' : row['author_id'],
    'country_id':c_id,
    'parent_id': row['parent_id'],
    'neg': row['neg'],
    'neu': row['neu'],
    'pos': row['pos'],
    'compound': row['compound'],
    'author': {
        "name": row['name'],
        "screen_name": row['screen_name'],
        "description": row['neg'],
    },
    "hashtags" : row['used_hashtags']
    }
    bulk_.append(doc)
    
bulk(es, bulk_)

(4551, [])

## Uloha 10

Bod c. 4 som vynechal, pretoze som s mentions nijako nepracoval. Podobne som pri povodne importe neselectol z databazy statuses_count (z tabulky accounts) a teda neviem pracovat s touto hodnotou. Avsak, podmienka sa nachadza zakomentovana v kode nizsie

In [11]:
es.search(index ="my_index", query = {
    "function_score": {
        "query": {
            "bool": {
                "must": {
                    "bool": {
                        "should": [
                            {
                                "match": {
                                    "author.name": {
                                        "query": "gates s0ros vaccine micr0chip",
                                        "fuzziness": "AUTO",
                                        "analyzer": "custom_shingles",
                                        "boost": 6.0
                                    }
                                }
                            },
                            {
                                "match": {
                                    "content": {
                                        "query": "gates s0ros vaccine micr0chip",
                                        "fuzziness": "AUTO",
                                        "analyzer": "englando",
                                        "boost": 8.0
                                    }
                                }
                            },
                            {
                                "match": {
                                    "author.description": {
                                        "query": "gates s0ros vaccine micr0chip",
                                        "fuzziness": "AUTO",
                                        "analyzer": "custom_shingles",
                                        "boost": 6.0
                                    }
                                }
                            },
                            {
                                "match": {
                                    "author.screen_name": {
                                        "query": "gates s0ros vaccine micr0chip",
                                        "fuzziness": "AUTO",
                                        "analyzer": "custom_ngram",
                                        "boost": 10.0
                                    }
                                }
                            }
                        ]
                    }
                },
                "filter": [
#                     "range": {
#                         "author.statuses_count": {
#                             "gt": 1000
#                         }
#                     },
                    {
                        "term": {
                            "hashtags": "qanon"
                        }
                    }
                ]
            }
        },
        "functions": [
            {
                "filter": {
                    "range": {
                        "retweet_count": {
                            "gte": 100,
                            "lte": 500
                        },
                    }
                },
                "weight": 6
            },
            {
                "filter": {
                    "range": {
                        "author.followers_count": {
                            "gt": 100
                        }
                    }
                },
                "weight": 3
            }
        ]
    }
}, aggs = {
    "hashtags":{
        "terms":{
            "field": "hashtags",
            "size": 100000
        }
    }   
})

{'took': 2456,
 'timed_out': False,
 '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 519.6272,
  'hits': [{'_index': 'my_index',
    '_type': '_doc',
    '_id': 'xJCnr30B6vb7fMH_l2YK',
    '_score': 519.6272,
    '_source': {'id': '1246435098951639041',
     'content': 'Gates Is Part Of A Covert Medical Cabal. Their Most Effective Tools For Human Depopulation?\r\n\r\nVaccines Spiked With Sterilization Chemical And Genetically Engineered Viruses With High Mortality Rate, Engineered To Target Specific Races And Genetic Profiles.\r\n\r\n#QAnon #Vaccine @potus https://t.co/H0QtpJDTq0',
     'location': None,
     'retweet_count': 124,
     'favorite_count': 0,
     'happened_at': '2020-04-04 13:50:59',
     'author_id': 3106728217,
     'country_id': -1,
     'parent_id': None,
     'neg': 0.0,
     'neu': 0.912,
     'pos': 0.088,
     'compound': 0.5256,
     'author': {'name': 'vee', 'screen_name

## Uloha 11

Pokusal som sa namiesto match_all pridat:

"query": {
    "match": {
        "hashtags": tags
    }
}

a dalsie podobne zapisy, ale vzdy som dostal naspat 0 hits, takze som nakoniec pouzil match_all

In [13]:
tags = "DeepstateVirus DeepStateFauci DeepStateVaccine QAnon Agenda21 CCPVirus ClimateChangeHoax GlobalWarmingHoax ChinaLiedPeopleDied SorosVirus 5GCoronavirus MAGA WWG1WGA Chemtrails flatEarth MoonLandingHoax moonhoax illuminati pizzaGateIsReal PedoGateIsReal 911truth 911insidejob reptilians"

In [23]:
es.search(index ="my_index", query = {
    "match_all": {}
}, aggs = {
    "weeks": {
        "date_histogram": {
            "field": "happened_at",
            "calendar_interval": "week"
        },
        "aggs": {
            "retweets_count": {
                "sum": {
                    "field": "retweet_count"
                }
            }
        }
    }
 })

C:\Users\Marek\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  from ipykernel import kernelapp as app


{'took': 54,
 'timed_out': False,
 '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'weeks': {'buckets': []}}}